## Part 1. Get metadata

In [14]:
import re
import sys
import pandas as pd
import urllib2 
from bs4 import BeautifulSoup 
import json
import requests

In [113]:
# change key search word
# search_word="information+privacy"

In [74]:
# change key search word
search_word="online retailing"
words=search_word.split()
kwd=str()
for w in (0,len(words)-1):
    if w==0:
        kwd=str(words[w])
    else:
        kwd=kwd+"+"+str(words[w])
print kwd

online+retailing


In [75]:
# Get keywords for each paper
url="https://misq.org/catalogsearch/result/index/?q={}&order=misq_year&dir=desc&limit=all".format(kwd)
# kw=pd.DataFrame(columns=['Keywords'])

In [76]:
url

'https://misq.org/catalogsearch/result/index/?q=online+retailing&order=misq_year&dir=desc&limit=all'

In [34]:
# set up browser name
from fake_useragent import UserAgent
ua = UserAgent()
# ua.chrome

In [35]:
header={'User-Agent': str(ua.chrome)}

In [77]:
# urlrr= requests.get(url, headers=header)
req = urllib2.Request(url, None, header)

In [78]:
# req = urllib2.Request(url, None, header)
urlo = urllib2.urlopen(req)
content = urlo.read()
bs= BeautifulSoup(content,"lxml")

In [79]:
i=0
df0=pd.DataFrame(columns=['Author', 'Title','Pub_type', 'Journal', 'Year_publish', 'Web_link','Abstract','Keywords','Doi','Category'])

In [80]:
long=len(bs.findAll("div", {"class": "product-shop"}))

In [81]:
long

2

In [82]:
for i in range(0,long):
    ## find link to paper abstract - ??instead of 0 should be i in cycle
    gg=bs.findAll("div", {"class": "product-shop"})[i]('a')
    # web_link=gg[0]['href']
    df0['Web_link'].loc[i]=gg[0]['href']
    # title=gg[0].text
    df0['Title'].loc[i]=gg[0].text
    # authors=bs.findAll("p", {"class": "author"})[0].text
    df0['Author'].loc[i]=bs.findAll("p", {"class": "author"})[i].text
    # get abstract
    reqq = urllib2.Request(df0['Web_link'].loc[i], None, header)
    url2 = urllib2.urlopen(reqq, timeout=10)
    content2 = url2.read()
    bs2= BeautifulSoup(content2,"lxml")
    # abstract=bs2.findAll("div", {"class": "product-specs"})[0].text
    df0['Abstract'].loc[i]=bs2.findAll("div", {"class": "product-specs"})[0].text.strip('\n')
    kwd=bs2.findAll("div", {"class": "collateral-box"})[2]
    table=kwd.findAll('table')
    # keywords, year, doi from table
    kwd=bs2.findAll("div", {"class": "collateral-box"})[2]
    table=kwd.findAll('table')[0]
    df=pd.read_html(str(table))[0]
    # year=df.iat[1,1]
    df0['Year_publish'].loc[i]=df.iat[1,1]
    # kwds=df[1][df[0]=="Keywords"]
    # kwds=df.iat[4,1]
    df0['Keywords'].loc[i]=df.iat[4,1]
    # doi=df[1][df[0]=="Page Numbers"]
    doi=df.iat[5,1]
    # doi_link=doi[doi.find(':')+2:]
    df0['Doi'].loc[i]=doi[doi.find(':')+2:]


In [83]:
df0 = pd.concat([df0['Author'],  df0['Title'],df0['Pub_type'],df0['Journal'],df0['Year_publish'],df0['Web_link'],df0['Abstract'],df0['Keywords'],df0['Doi'],df0['Category']], axis=1)

In [84]:
df0['Pub_type']='Journal Article'
df0['Journal']='MISQ'
df0['Category']=search_word

In [85]:
df0['Author']=df0['Author'].str.encode('ascii','ignore')
df0['Title']=df0['Title'].str.encode('ascii','ignore')
df0['Web_link']=df0['Web_link'].str.encode('ascii','ignore')
df0['Abstract']=df0['Abstract'].str.encode('ascii','ignore')
df0['Keywords']=df0['Keywords'].str.encode('ascii','ignore')
# df0['Title']=df0['Title'].str.encode('ascii','ignore')

In [93]:
df0.head()

,Author,Title,Pub_type,Journal,Year_publish,Web_link,Abstract,Keywords,Doi,Category
0,"Author: Young Kwark, Jianqing Chen, and Sriniv...",Platform or Wholesale? A Strategic Tool for O...,Journal Article,MISQ,2017,https://misq.org/platform-or-wholesale-a-strat...,Online retailing is dominated by a channel str...,"Third-party information, pricing scheme, onlin...",10.25300/MISQ/2017/41.3.05,online retailing


In [87]:
df0=df0[df0["Year_publish"].astype(int)>=2012]
len(df0)

1

In [88]:
# export to csv file
df0.to_csv('List_MISQ_'+search_word+'.csv')